In [3]:
import csv
import re
class Employee:
    #Create an Employee with these info
    def __init__(self, ID, Name, Position, Salary, Email):
        self.ID = ID
        self.Name = Name
        self.Position = Position
        self.Salary = Salary
        self.Email = Email

    def update_details(self, **k):
        for key, value in k.items():
            if hasattr(self, key):
                if key == "ID":
                    print("Warning!! You cannot change the ID.")
                elif key == "Name":
                    self.Name = value
                elif key == "Position":
                    self.Position = value
                elif key == "Salary":
                    self.Salary = value
                elif key == "Email":
                    self.Email = value
            else:
                print(f"Invalid attribute: {key}")

    def display(self):
        print("ID: {}, Name: {}, Position: {}, Salary: {}, Email: {}".format(self.ID,self.Name,self.Position,self.Salary,self.Email))

In [6]:
class EmployeeManager:
    def __init__(self, file="employees.csv"):
        self.file = file
        self.employees = []
        self.__load_employees_from_file()

    def __load_employees_from_file(self):
        try:
            with open(self.file, mode="r") as file:
                text = csv.DictReader(file) #better than basic read functions as it use headers as key for the values
                for row in text:
                    self.employees.append(
                        Employee(
                            ID=int(row["ID"]),
                            Name=row["Name"],
                            Position=row["Position"],
                            Salary=float(row["Salary"]),
                            Email=row["Email"]
                        )
                    )
            print("Employees loaded successfully.")
        except FileNotFoundError:
            print("Warning: {} not found. Starting with an empty list.".format(self.filename))
        except Exception as e:
            print("Error while loading employees: {}".format(e))
    #Save any modification happens for the employees info
    def __save_employee_data(self):
        try:
            with open(self.file, mode="w", newline="") as file:
                Datawriting = csv.DictWriter(file, fieldnames=["ID", "Name", "Position", "Salary", "Email"])#headers for the file
                Datawriting.writeheader()
                for emp in self.employees:
                    Datawriting.writerow({
                        "ID": emp.ID,
                        "Name": emp.Name,
                        "Position": emp.Position,
                        "Salary": emp.Salary,
                        "Email": emp.Email
                    })
            #print("Employees saved successfully.")
        except Exception as e:
            print("Error while saving employees: {}".format(e))
    # validate the entered info
    def __data_validation(self, employee):
        if not isinstance(employee.Salary, (int, float)) or employee.Salary <= 0:
            print("Error! Salary must be a positive numeric value.")
            return False

        if not self.__is_valid_email(employee.Email):
            print("Error! Please enter a valid email address.")
            return False

        print("Validation successful!")
        return True

    def __is_valid_email(self, email):
       pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
       if re.match(pattern, email) is None:
            return False
       valid_email_domains = ["mail", "gmail", "yahoo"]
       domain = email.split('@')[-1].split('.')[0].lower()
       if domain not in valid_email_domains:
            return False

       return True


    def add_employee(self, employee):
        if not self.__data_validation(employee):
            print("Employee data is invalid. Employee not added.")
            return
        if any(emp.ID == employee.ID for emp in self.employees):
            print("Error! Employee with this ID already exists.")
        else:
            self.employees.append(employee)
            self.__save_employee_data()
            print("Employee {} added successfully.".format(employee.Name))

    def update_employee(self, ID, **k):
        employee = self.find_employee(ID)
        if employee:
            temp_employee = Employee(
               ID=employee.ID,  # Keep the same ID
               Name=k.get("Name", employee.Name),
               Position=k.get("Position", employee.Position),
               Salary=k.get("Salary", employee.Salary),
               Email=k.get("Email", employee.Email)
            )
            if not self.__data_validation(temp_employee):
               print("Employee update data is invalid. Update aborted.")
               return
            employee.update_details(**k)
            self.__save_employee_data()
            #print("Employee with ID {} updated successfully.".format(ID))
        else:
            print("Error: No employee with ID {} found.".format(ID))
    def delete_employee(self,ID):
        emp=self.find_employee(ID)
        if emp:
            self.employees = [emp for emp in self.employees if emp.ID != ID]
            self.__save_employee_data()
            #print(f"Employee with ID {ID} deleted successfully.")
        else:
            print(f"No employee found with ID {ID}.")

    def find_employee(self, ID):
        for emp in self.employees:
            if emp.ID == ID:
                return emp
        return None

    def list_employees(self):
        if not self.employees:
            print("No employees in the system.")
        else:
            print("Employee Info:")
            for emp in self.employees:
                emp.display()
    def run(self):
        #function for displaying main menu
        while True:
           print("===============================")
           print("\nEmployee Management System")
           print("===============================")
           print("1. Add Employee")
           print("2. Update Employee")
           print("3. Delete Employee")
           print("4. List Employees")
           print("5. Search Employee")
           print("6. Exit")
           print("===============================")

           choice = input("Choose an option: ")

           if choice == "1":
               try:
                   ID = int(input("Enter Employee ID: "))
                   Name = input("Enter Employee Name: ")
                   Position = input("Enter Employee Position: ")
                   Salary = float(input("Enter Employee Salary: "))
                   Email = input("Enter Employee Email: ")

                   new_employee = Employee(ID=ID, Name=Name, Position=Position, Salary=Salary, Email=Email)
                   self.add_employee(new_employee)
               except ValueError:
                    print("Error: Invalid input. Please enter valid data.")

           elif choice == "2":
                try:
                   ID = int(input("Enter Employee ID to update: "))
                   print("Leave fields blank if you don't want to update them.")
                   Name = input("Enter new name (or press Enter to skip): ")
                   Position = input("Enter new position (or press Enter to skip): ")
                   Salary = input("Enter new salary (or press Enter to skip): ")
                   Email = input("Enter new email (or press Enter to skip): ")

                   kwargs = {}
                   if Name:
                       kwargs["Name"] = Name
                   if Position:
                       kwargs["Position"] = Position
                   if Salary:
                       try:
                           kwargs["Salary"] = float(Salary)
                       except ValueError:
                            print("Error: Salary must be a numeric value.")
                            continue
                   if Email:
                        kwargs["Email"] = Email

                   self.update_employee(ID, **kwargs)
                except ValueError:
                    print("Error: Invalid input. Please enter a valid ID.")

           elif choice == "3":
               try:
                   ID = int(input("Enter Employee ID to delete: "))
                   self.delete_employee(ID)
               except ValueError:
                    print("Error: Invalid input. Please enter a valid ID.")

           elif choice == "4":
                self.list_employees()

           elif choice == "5":
               try:
                   ID = int(input("Enter Employee ID to search: "))
                   emp = self.find_employee(ID)
                   if emp:
                      emp.display()
                   else:
                      print(f"No employee found with ID {ID}.")
               except ValueError:
                   print("Error: Invalid input. Please enter a valid ID.")

           elif choice == "6":
            # Exiting the system
                print("Exiting the system. Goodbye!")
                break

           else:
                print("Invalid choice. Please try again.")

In [7]:
if __name__ == "__main__":
    manager = EmployeeManager()
    manager.run()

Employees loaded successfully.

Employee Management System
1. Add Employee
2. Update Employee
3. Delete Employee
4. List Employees
5. Search Employee
6. Exit
Exiting the system. Goodbye!
